In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle

from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from random import uniform
from scipy.stats import randint
from sklearn.linear_model import Lasso
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [2]:
ball_release = pd.read_csv(r"C:\Users\ASUS\OneDrive\Documents\Academics\Summer Semester\Dataset\tossstartframes.csv",header=None)
data_path = r"C:\Users\ASUS\OneDrive\Documents\Academics\Summer Semester\Dataset\CSVs\\"
#test_data = pd.read_excel(r"C:\Users\ASUS\OneDrive\Documents\Academics\Summer Semester\Dataset\model_inout\test_25-06.xlsx")
frames = pd.read_csv(r"C:\Users\ASUS\OneDrive\Documents\Academics\Summer Semester\Dataset\frames.csv")
cont_frames = pd.read_csv(r"C:\Users\ASUS\OneDrive\Documents\Academics\Summer Semester\Dataset\contactframes.csv")
ball_toss = pd.read_csv(r"C:\Users\ASUS\OneDrive\Documents\Academics\Summer Semester\Dataset\tossstartframes.csv")

In [3]:
def classify_data(range_value,file,data_path,bl_toss):
    data_list = []
    class_list = []
    ball_toss = []

    for i in range(len(frames)):
            #print(i)
            file_name = frames.iloc[i][0]
            toss_row = frames.iloc[i][1]
            hit_row = frames.iloc[i][2]

            for csv in os.listdir(data_path):

                if file_name in csv:
                        #print("\n",file_name)
                        data = pd.read_csv(data_path+"\\"+csv)
                        #print(data)
                        class_list = ["Before" for i in range(0,hit_row-range_value)]
                        class_list.extend(["Hit" for i in range(hit_row-range_value,hit_row+range_value)])
                        class_list.extend(["After" for i in range(hit_row+range_value,len(data))])

                        ball_toss = ["Before Toss" for i in range(0,toss_row-range_value)]
                        ball_toss.extend(["Toss" for i in range(toss_row-range_value,toss_row+range_value)])
                        ball_toss.extend(["After Toss" for i in range(toss_row+range_value,len(data))])

                        file_list = [file_name for i in range(len(data))]
                        data = data.assign(File=file_list,Class_1=class_list,Class_2=ball_toss)
                        data_list.append(data)   
                        dataframe = pd.concat(data_list)
            #print(len(class_list),len(ball_toss),len(file_list)) 
    return dataframe

In [4]:
def cal_ang(model_data):
    
    
    angle_rad = []
    sho_rand_angle = []
    
    for x1,y1,x2,y2 in zip(model_data["NOSE_X"],model_data['NOSE_Y'],model_data['RIGHT_WRIST_X'],model_data['RIGHT_WRIST_Y']):
        
        
    
        angle_rad.append(math.atan2( y2 - y1 , x2 - x1 ))
        #angle_deg = math.degrees(angle_rad)
        #nose_wrist_angle.append(pygame.math.Vector2(x1, y1).angle_to((x2, y2)))
    
    for x1,y1,x2,y2 in zip(model_data["LEFT_WRIST_X"],model_data['LEFT_WRIST_Y'],model_data['RIGHT_SHOULDER_X'],model_data['RIGHT_SHOULDER_Y']):
        
        #print(slope1,slope2)
        sho_rand_angle.append(math.atan2( y2 - y1 , x2 - x1 ))
        
    return angle_rad,sho_rand_angle


In [5]:
model_data = classify_data(12,cont_frames,data_path,ball_toss)

#model_data = model_data.drop(['Class'], axis=1)
#model_data = model_data.drop(['Unnamed: 51'], axis=1)

nose_angle, sho_angle = cal_ang(model_data)

model_data['Nose-Wrist Angle'] = nose_angle 
model_data['Sho-Wrist Angle'] = sho_angle

In [6]:
model_data.to_csv(r"C:\Users\ASUS\OneDrive\Documents\Academics\Summer Semester\Dataset\model_inout\DataSet_01_07.csv")

In [6]:
model_data = model_data.drop(['Unnamed: 51'], axis=1)
#model_data = model_data.drop(['Unnamed: 56'], axis=1)
#model_data = model_data.drop(['Unnamed: 57'], axis=1)
#model_data = model_data.drop(['Unnamed: 58'], axis=1)
#model_data = model_data.drop(['Unnamed: 0'], axis=1)

In [7]:
nose_df = pd.DataFrame(nose_angle)
sho_df = pd.DataFrame(sho_angle)
#nose_df.to_csv(r"C:\Users\ASUS\OneDrive\Documents\Academics\Summer Semester\noseeangle.csv")
#sho_df.to_csv(r"C:\Users\ASUS\OneDrive\Documents\Academics\Summer Semester\shoanglee.csv")

In [ ]:
#df_mod_dat = model_data.iloc[569:]
#test_data = model_data[:568]

#df1 = model_data.iloc[:568]
#df2 = model_data.iloc[1298:]
#df1 = model_data.iloc[579:1299]

#frames = [df1, df2]

#df_mod_dat = pd.concat(frames)
#test_data = model_data.iloc[569:1299]

In [9]:
feature_columns = ['NOSE_X', 'NOSE_Y', 'NOSE_Z', 'LEFT_SHOULDER_X', 'LEFT_SHOULDER_Y',
       'LEFT_SHOULDER_Z', 'RIGHT_SHOULDER_X', 'RIGHT_SHOULDER_Y',
       'RIGHT_SHOULDER_Z', 'LEFT_ELBOW_X', 'LEFT_ELBOW_Y', 'LEFT_ELBOW_Z',
       'RIGHT_ELBOW_X', 'RIGHT_ELBOW_Y', 'RIGHT_ELBOW_Z', 'LEFT_WRIST_X',
       'LEFT_WRIST_Y', 'LEFT_WRIST_Z', 'RIGHT_WRIST_X', 'RIGHT_WRIST_Y',
       'RIGHT_WRIST_Z', 'LEFT_HIP_X', 'LEFT_HIP_Y', 'LEFT_HIP_Z',
       'RIGHT_HIP_X', 'RIGHT_HIP_Y', 'RIGHT_HIP_Z', 'LEFT_KNEE_X',
       'LEFT_KNEE_Y', 'LEFT_KNEE_Z', 'RIGHT_KNEE_X', 'RIGHT_KNEE_Y',
       'RIGHT_KNEE_Z', 'LEFT_ANKLE_X', 'LEFT_ANKLE_Y', 'LEFT_ANKLE_Z',
       'RIGHT_ANKLE_X', 'RIGHT_ANKLE_Y', 'RIGHT_ANKLE_Z', 'LEFT_HEEL_X',
       'LEFT_HEEL_Y', 'LEFT_HEEL_Z', 'RIGHT_HEEL_X', 'RIGHT_HEEL_Y',
       'RIGHT_HEEL_Z', 'LEFT_FOOT_INDEX_X', 'LEFT_FOOT_INDEX_Y',
       'LEFT_FOOT_INDEX_Z', 'RIGHT_FOOT_INDEX_X', 'RIGHT_FOOT_INDEX_Y',
       'RIGHT_FOOT_INDEX_Z','Sho-Wrist Angle']

In [10]:
X = model_data[feature_columns]
Y = model_data['Class_2']

label_encoder = preprocessing.LabelEncoder()
y = label_encoder.fit_transform(Y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [32]:
lasso = Lasso(alpha=0.1)

lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

coefficients = pd.DataFrame({'Features': X_train.columns, 'Coefficients': lasso.coef_})

coef_lis =[]

for j in range(len(coefficients)):
    if coefficients.iloc[j][1]!= 0:
        coef_lis.append((coefficients.iloc[j][0]))
        
coef_lis.append('Sho-Wrist Angle')
        
    
#print(([i for i in coefficients['Coefficients'] if i!=0]))


Mean Squared Error: 0.06465233953607206


C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.995e+00, tolerance: 6.176e-01
  model = cd_fast.enet_coordinate_descent(


In [33]:
coef_lis

['NOSE_Z',
 'LEFT_SHOULDER_Y',
 'LEFT_SHOULDER_Z',
 'RIGHT_SHOULDER_X',
 'LEFT_ELBOW_Y',
 'LEFT_ELBOW_Z',
 'RIGHT_ELBOW_Y',
 'RIGHT_ELBOW_Z',
 'LEFT_WRIST_X',
 'LEFT_WRIST_Y',
 'LEFT_WRIST_Z',
 'RIGHT_WRIST_X',
 'RIGHT_WRIST_Z',
 'RIGHT_HIP_X',
 'LEFT_KNEE_X',
 'LEFT_KNEE_Z',
 'RIGHT_KNEE_X',
 'RIGHT_KNEE_Y',
 'RIGHT_KNEE_Z',
 'RIGHT_ANKLE_X',
 'LEFT_HEEL_X',
 'LEFT_HEEL_Y',
 'LEFT_HEEL_Z',
 'RIGHT_FOOT_INDEX_Y',
 'RIGHT_FOOT_INDEX_Z',
 'Sho-Wrist Angle']

In [34]:
new_X = model_data[coef_lis]
new_X_train, new_X_test, y_train, y_test = train_test_split(new_X, y, test_size=0.3, random_state=42)

Random Search for XGModel Class 1

In [35]:
param_grid = {'silent': [False],'max_depth': [6, 10, 15, 20],'learning_rate': [0.001, 0.01, 0.1, 0.2, 0,3],'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],'gamma': [0, 0.25, 0.5, 1.0],'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],'n_estimators': [100]}

In [ ]:
#fit_params = {'eval_metric': 'mlogloss','early_stopping_rounds': 10,'eval_set': [(new_X_train, y_train)]}

NameError: name 'y_train' is not defined

In [36]:
# fit model no training data
xg_model = XGBClassifier()
xg_model.fit(new_X_train, y_train) 

new_y_pred = xg_model.predict(new_X_test)

accuracy = accuracy_score(y_test, new_y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9983515836571294


In [39]:
folds = 5
param_comb = 100

skf = StratifiedKFo ld(n_splits=folds, shuffle = True, random_state = 42)

random_search = RandomizedSearchCV(xg_model, param_distributions=param_grid, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X_train,y_train), verbose=3, random_state=42 )

random_search.fit(new_X_train, y_train)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


[10:45:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "silent" } are not used.



RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x000002990DCD4E40>,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_...
                                        'colsample_bytree': [0.4, 0.5, 0.6, 0.7,
                                                             0.8, 0.9, 1.0],
                                        'gamma': [0, 0.25, 0.5, 1.0],
                                        'learning_rate': [0.001, 0.01, 0.1, 0.2,
                                                          0, 3],
                                        'max_depth': [6, 10, 15, 20],
                                        'min_child_weight': [0.5, 1.0, 3.0, 5.0,
                                                             7.0, 10.0],
                                        'n_estimators': [100],
                                        'reg_lambda': [0.1, 1.0, 5.0, 10.0,
                                                       50.0, 100.0],
                                        'silent': [False],
                                        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9,
                                                      1.0]},
                   random_state=42, scoring='roc_auc', verbose=3)

In [15]:
print(random_search.best_params_) 

{'subsample': 0.7, 'silent': False, 'reg_lambda': 50.0, 'n_estimators': 100, 'min_child_weight': 5.0, 'max_depth': 6, 'learning_rate': 0.2, 'gamma': 1.0, 'colsample_bytree': 0.9, 'colsample_bylevel': 0.4}


In [17]:
new_y_pred = random_search.predict(new_X_test)

accuracy = accuracy_score(y_test, new_y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9983515836571294


In [ ]:
xg_model = XGBClassifier()
xg_model.fit(new_X_train, y_train) 

new_y_pred = xg_model.predict(new_X_test)

accuracy = accuracy_score(y_test, new_y_pred)
print("Accuracy:", accuracy)


In [ ]:
X_pred = test_data[coef_lis]

y_rand_test = test_model_rand.predict(X_pred)

xg_data_f = pd.DataFrame(y_rand_test)
xg_data_f.to_csv(r"C:\Users\ASUS\OneDrive\Documents\Academics\Summer Semester\Dataset\model_inout\rand_prediction_kev2_hit_10_7.csv")

In [ ]:
X_pred = test_data[coef_lis]

y_rand_test = xg_model.predict(X_pred)

xg_data_f = pd.DataFrame(y_rand_test)
xg_data_f.to_csv(r"C:\Users\ASUS\OneDrive\Documents\Academics\Summer Semester\Dataset\model_inout\xg_prediction_toss_finetuned_7_7.csv")

In [ ]:
eval_set = [(new_X_train, y_train), (new_X_test, y_test)]
eval_metric = ["auc","error"]
%time model.fit(X_train, y_train, eval_metric=eval_metric, eval_set=eval_set, verbose=True)

**Random Forest**

In [ ]:
test_model_rand = RandomForestClassifier()

test_model_rand.fit(new_X_train, y_train)
new_y_pred = test_model_rand.predict(new_X_test)

accuracy = accuracy_score(y_test, new_y_pred)
print("Accuracy:", accuracy)



In [ ]:
X_pred = test_data[coef_lis]

y_rand_test = test_model_rand.predict(X_pred)
data_f = pd.DataFrame(y_rand_test)

In [ ]:
data_f.to_csv(r"C:\Users\ASUS\OneDrive\Documents\Academics\Summer Semester\Dataset\model_inout\")

In [40]:
model_filename = 'RandomXGB_Toss_01_08.sav'
pickle.dump(xg_model, open(model_filename, 'wb'))

loaded_model = pickle.load(open(model_filename, 'rb'))
result = loaded_model.score(new_X_test, y_test)
print(result)

0.9983515836571294


In [ ]:
import io
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder


In [2]:
raw_data = pd.read_csv(r"C:\Users\ASUS\OneDrive\Documents\Academics\Summer Semester\Dataset\model_inout\DataSet_10_07.csv")

NameError: name 'pd' is not defined

In [ ]:
raw_data.columns

In [ ]:
data = raw_data[hit_toss].copy()

In [ ]:
data.to_csv("Hit_Toss_Dataset_26_07.csv")

In [ ]:
encoder = LabelEncoder()
data["Class_1"] = encoder.fit_transform(data["Class_1"])
data["Class_2"] = encoder.fit_transform(data["Class_2"])
data["Out"] = encoder.fit_transform(data["Out"])

In [ ]:
#fet = ['NOSE_X','NOSE_Y','NOSE_Z','LEFT_SHOULDER_X','LEFT_SHOULDER_Y','RIGHT_SHOULDER_X','RIGHT_SHOULDER_Z','LEFT_ELBOW_Y','LEFT_ELBOW_Z','RIGHT_ELBOW_X','RIGHT_ELBOW_Z','LEFT_WRIST_Y','LEFT_WRIST_Z','RIGHT_WRIST_X','RIGHT_WRIST_Y','RIGHT_HIP_X','LEFT_KNEE_Z','RIGHT_KNEE_X','RIGHT_ANKLE_Z','LEFT_HEEL_Y','RIGHT_HEEL_X','RIGHT_HEEL_Y','LEFT_FOOT_INDEX_X','LEFT_FOOT_INDEX_Z','Nose-Wrist Angle','Sho-Wrist Angle','Class_1','Class_2','Out']
fet = ['NOSE_X','NOSE_Y','LEFT_SHOULDER_X','LEFT_SHOULDER_Y','RIGHT_SHOULDER_X','LEFT_ELBOW_Y','LEFT_ELBOW_Z','RIGHT_ELBOW_X','RIGHT_ELBOW_Z','LEFT_WRIST_Y','LEFT_WRIST_Z','RIGHT_WRIST_Y','RIGHT_HIP_X','RIGHT_KNEE_X','LEFT_HEEL_Y','RIGHT_HEEL_X','RIGHT_HEEL_Y','LEFT_FOOT_INDEX_X','Nose-Wrist Angle','Sho-Wrist Angle','Class_1','Class_2','Out']

In [ ]:
#cam3 slow ---- Hit and Toss only
df1 = data.iloc[:288]
df2 = data.iloc[336:]
frames = [df1,df2]
dataframe = pd.concat(frames)
test = data.iloc[289:336]

In [ ]:
#raw data
#df1 = data.iloc[:7539]
#df2 = data.iloc[8441:]
#frames = [df1,df2]
#dataframe = pd.concat(frames)
#test = data.iloc[7540:8440]

In [ ]:
#Kev 1 data
dataframe = data.iloc[48:]
test = data.iloc[:47]

In [ ]:
dataframe = dataframe[fet]
test = test[fet]

In [ ]:
cols = dataframe.columns[:-1]

In [ ]:
dataframe

In [ ]:
features = len(cols)

In [ ]:
#model = Sequential()
#model.add(LSTM(100, activation = 'tanh',return_sequences = True, input_shape = (1,features)))
#model.add(LSTM(40, activation = 'relu'))
#model.add(Dense(1, activation = 'sigmoid'))
#model.compile(optimizer = 'AdaGrad', loss = 'binary_crossentropy' ,metrics = ['accuracy'])
# proff = 46 percent


#model = Sequential()
#model.add(LSTM(100, activation = 'tanh',return_sequences = True, input_shape = (1,features)))
#model.add(LSTM(80, activation = 'relu',return_sequences = True))
#model.add(LSTM(60, activation = 'tanh'))
#model.add(Dense(1, activation = 'sigmoid'))
#model.compile(optimizer = 'AdaGrad', loss = 'binary_crossentropy' ,metrics = ['accuracy'])


#model = Sequential()
#model.add(LSTM(100, activation = 'tanh',return_sequences = True, input_shape = (1,features)))
#model.add(LSTM(80, activation = 'relu',return_sequences = True))
#model.add(LSTM(60, activation = 'swish'))

#model.add(Dense(1, activation = 'sigmoid'))
#model.compile(optimizer = 'AdaGrad', loss = 'binary_crossentropy' ,metrics = ['accuracy'])

#model = Sequential()
#model.add(LSTM(100, activation = 'sigmoid',return_sequences = True, input_shape = (1,features)))
#model.add(LSTM(60, activation = 'sigmoid',return_sequences = True))
#model.add(LSTM(40, activation = 'sigmoid'))

#model.add(Dense(1, activation = 'sigmoid'))
#model.compile(optimizer = 'AdaGrad', loss = 'binary_crossentropy' ,metrics = ['accuracy'])
#== best var.. unpro == 23 and pro==48


#model = Sequential()
#model.add(LSTM(100, activation = 'relu',return_sequences = True, input_shape = (1,features)))
#model.add(LSTM(60, activation = 'relu',return_sequences = True))
#model.add(LSTM(40, activation = 'sigmoid'))

#model.add(Dense(1, activation = 'sigmoid'))
#model.compile(optimizer = 'AdaGrad', loss = 'binary_crossentropy' ,metrics = ['accuracy'])
#unpro = 36 and pro >50 

#model = Sequential()
#model.add(LSTM(100, activation = 'sigmoid',return_sequences = True, input_shape = (1,features)))
#model.add(LSTM(60, activation = 'tanh',return_sequences = True))
#model.add(LSTM(40, activation = 'sigmoid'))

#model.add(Dense(1, activation = 'sigmoid'))
#model.compile(optimizer = 'Adam', loss = 'binary_crossentropy' ,metrics = ['accuracy'])
# unpro+2 = pro percent-- Sensible model

#model = Sequential()
#model.add(LSTM(100, activation = 'tanh',return_sequences = True, input_shape = (1,features)))
#model.add(LSTM(60, activation = 'relu',return_sequences = True))
#model.add(LSTM(40, activation = 'swish'))

#model.add(Dense(1, activation = 'sigmoid'))
#model.compile(optimizer = 'Adam', loss = 'binary_crossentropy' ,metrics = ['accuracy'])
# highest accuracy = 0.8837. but percent of pro similar to unpro

In [ ]:
model = Sequential()
model.add(LSTM(100, activation = 'tanh',return_sequences = True, input_shape = (1,features)))
model.add(LSTM(80, activation = 'relu',return_sequences = True))
model.add(LSTM(60, activation = 'swish'))

model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer = 'Adam', loss = 'binary_crossentropy' ,metrics = ['accuracy'])

In [ ]:
X = dataframe[fet[:-1]]
X = np.resize(X,(X.shape[0],1,X.shape[1]))  
y = dataframe[fet[-1]]
model.fit(X,y,batch_size= len(X),epochs =20)

In [1]:
y

NameError: name 'y' is not defined

In [ ]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
pred_test = test[test.columns[:-1]]
pred_test["Class_1"] = encoder.fit_transform(pred_test["Class_1"])
pred_test["Class_2"] = encoder.fit_transform(pred_test["Class_2"])
X_pred = np.resize(pred_test,(pred_test.shape[0],1,pred_test.shape[1]))

In [18]:
import pandas as pd
import pickle

In [19]:
data = pd.read_csv(r"C:\Users\ASUS\OneDrive\Documents\Academics\Summer Semester\Dataset\CSVs\Kevflipped3.csv")

In [24]:
data.columns

Index(['Unnamed: 0', 'NOSE_X', 'NOSE_Y', 'NOSE_Z', 'LEFT_SHOULDER_X',
       'LEFT_SHOULDER_Y', 'LEFT_SHOULDER_Z', 'RIGHT_SHOULDER_X',
       'RIGHT_SHOULDER_Y', 'RIGHT_SHOULDER_Z', 'LEFT_ELBOW_X', 'LEFT_ELBOW_Y',
       'LEFT_ELBOW_Z', 'RIGHT_ELBOW_X', 'RIGHT_ELBOW_Y', 'RIGHT_ELBOW_Z',
       'LEFT_WRIST_X', 'LEFT_WRIST_Y', 'LEFT_WRIST_Z', 'RIGHT_WRIST_X',
       'RIGHT_WRIST_Y', 'RIGHT_WRIST_Z', 'LEFT_HIP_X', 'LEFT_HIP_Y',
       'LEFT_HIP_Z', 'RIGHT_HIP_X', 'RIGHT_HIP_Y', 'RIGHT_HIP_Z',
       'LEFT_KNEE_X', 'LEFT_KNEE_Y', 'LEFT_KNEE_Z', 'RIGHT_KNEE_X',
       'RIGHT_KNEE_Y', 'RIGHT_KNEE_Z', 'LEFT_ANKLE_X', 'LEFT_ANKLE_Y',
       'LEFT_ANKLE_Z', 'RIGHT_ANKLE_X', 'RIGHT_ANKLE_Y', 'RIGHT_ANKLE_Z',
       'LEFT_HEEL_X', 'LEFT_HEEL_Y', 'LEFT_HEEL_Z', 'RIGHT_HEEL_X',
       'RIGHT_HEEL_Y', 'RIGHT_HEEL_Z', 'LEFT_FOOT_INDEX_X',
       'LEFT_FOOT_INDEX_Y', 'LEFT_FOOT_INDEX_Z', 'RIGHT_FOOT_INDEX_X',
       'RIGHT_FOOT_INDEX_Y', 'RIGHT_FOOT_INDEX_Z', 'Nose-Wrist Angle',
       'Sho-Wrist Angl

In [23]:
da_fr = cal_ang(data)
data["Nose-Wrist Angle"] = da_fr[0]
data["Sho-Wrist Angle"] = da_fr[1]

In [53]:
data.to_csv(r"C:\Users\ASUS\OneDrive\Documents\Academics\Summer Semester\Dataset\Kevflipped3.csv")

In [83]:
pickled_model = pickle.load(open(r"C:\Users\ASUS\OneDrive\Documents\Academics\Summer Semester\Hit Classifier Models\XGBoost_Hit_01_07.sav", 'rb'))

In [84]:
fet = pickled_model.get_booster().feature_names

In [85]:
output = pickled_model.predict(data[fet])

In [78]:
th = pred_check(output)

In [77]:
def pred_check(arr):
    if arr[0] != 0:
        arr[0] = 0

    for i in range(len(arr) - 1):
        if arr[i] == 0 and arr[i+1] == 1:
            arr[i+1] = 2
        if arr[i] == 2 and arr[i + 1] == 0:
            arr[i + 1] = 1
        if arr[i] == 1 and arr[i + 1] != 1:
            arr[i + 1] = 1

    
    return arr


In [25]:
df = pd.DataFrame(random_search.predict(data[coef_lis]))

In [28]:
random_search.predict(data[coef_lis])

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [79]:
th

array([0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [27]:
df.to_csv(r"C:\Users\ASUS\OneDrive\Documents\Academics\Summer Semester\Dataset\model_inout\OOOOOOOOOOOOOO.csv")